In [1]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [2]:
data = data.iloc[:3524]

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3524 entries, 0 to 3523
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    3524 non-null   object 
 1   open    3524 non-null   float64
 2   high    3524 non-null   float64
 3   low     3524 non-null   float64
 4   close   3524 non-null   float64
 5   volume  3524 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 165.3+ KB


### prepare models for close, high, low

In [4]:
from talib import RSI
data['rsi_14'] = RSI(data['close'], timeperiod=14)
#data['rsi_14'] = data['rsi_14'].shift(1)

from talib import MA, SMA, EMA, WMA
data['ma_9'] = MA(data['close'], timeperiod=9)
#data['ma_9'] = data['ma_9'].shift(1)
data['sma_9'] = SMA(data['close'], timeperiod=9)
#data['sma_9'] = data['sma_9'].shift(1)
data['wma_9'] = WMA(data['close'], timeperiod=9)
#data['wma_9'] = data['wma_9'].shift(1)

from talib import MACD
data['macd'], data['signal'], data['hist'] = MACD(data['close'])
#data['macd'] = data['macd'].shift(1)
#data['signal'] = data['signal'].shift(1)
#data['hist'] = data['hist'].shift(1)

from talib import ADX
data['adx'] = ADX(data['high'], data['low'], data['close'])
#data['adx'] = data['adx'].shift(1)

from talib import ATR
data['atr'] = ATR(high=data['high'], low=data['low'], close=data['close'], timeperiod=14)
#data['atr'] = data['atr'].shift(1)

from talib import SAR
data['sar'] = SAR(high=data['high'], low=data['low'], acceleration=0.02, maximum=0.2)
#data['sar'] = data['sar'].shift(1)

from talib import TEMA
data['tema'] = TEMA(data['close'], timeperiod=14)
#data['tema'] = data['tema'].shift(1)

from talib import ROC
data['roc'] = ROC(data['close'], timeperiod=14)
#data['roc'] = data['roc'].shift(1)

data.dropna(axis=0, inplace=True)

In [5]:
best_features_c = ('close', 'ma_9', 'sma_9', 'macd', 'signal', 'hist', 'adx', 'atr', 'sar', 'tema', 'roc')
best_features_h = ('high', 'ma_9', 'sma_9', 'macd', 'signal', 'hist', 'adx', 'atr', 'sar', 'tema', 'roc')
best_features_l = ('low', 'ma_9', 'sma_9', 'macd', 'signal', 'hist', 'adx', 'atr', 'sar', 'tema', 'roc')

In [6]:
data_input_c = data[list(best_features_c)]
data_target_c = data[['close']]

data_input_h = data[list(best_features_h)]
data_target_h = data[['high']]

data_input_l = data[list(best_features_l)]
data_target_l = data[['low']]

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler_c = MinMaxScaler()
data_scaled_c = scaler_c.fit_transform(data_input_c)

scaler_target_c = MinMaxScaler()
target_scaled_c = scaler_target_c.fit_transform(data_target_c)

scaler_h = MinMaxScaler()
data_scaled_h = scaler_h.fit_transform(data_input_h)

scaler_target_h = MinMaxScaler()
target_scaled_h = scaler_target_h.fit_transform(data_target_h)

scaler_l = MinMaxScaler()
data_scaled_l = scaler_l.fit_transform(data_input_l)

scaler_target_l = MinMaxScaler()
target_scaled_l = scaler_target_l.fit_transform(data_target_l)

In [8]:
seq_length = 14  # Number of time steps in each sequence

num_features_c = data_input_c.shape[1]
num_features_h = data_input_h.shape[1]
num_features_l = data_input_l.shape[1]

X_features_c = data_scaled_c
y_target_c = target_scaled_c
X_features_h = data_scaled_h
y_target_h = target_scaled_h
X_features_l = data_scaled_l
y_target_l = target_scaled_l

In [9]:
import numpy as np

# Create input sequences and targets
def create_sequences(features, target, seq_length):
    X_seq = []
    y_seq = []
    for i in range(len(features) - seq_length):
        X_seq.append(features[i:i+seq_length])  # Input sequence
        y_seq.append(target[i+seq_length]) # Target value (next data point)
    return np.array(X_seq), np.array(y_seq)

In [10]:
X_seq_c, y_seq_c = create_sequences(X_features_c, y_target_c, seq_length)
X_seq_h, y_seq_h = create_sequences(X_features_h, y_target_h, seq_length)
X_seq_l, y_seq_l = create_sequences(X_features_l, y_target_l, seq_length)

# Reshape X_seq to fit LSTM model input shape
X_seq_c = X_seq_c.reshape(X_seq_c.shape[0], seq_length, num_features_c)
X_seq_h = X_seq_h.reshape(X_seq_h.shape[0], seq_length, num_features_h)
X_seq_l = X_seq_l.reshape(X_seq_l.shape[0], seq_length, num_features_l)

In [11]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Build LSTM model
model_c = Sequential([
    LSTM(25, input_shape=(seq_length, num_features_c)),
    Dense(1)
])

# Compile the model
model_c.compile(optimizer='adam', loss='mean_squared_error')

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training the model with early stopping
model_c.fit(X_seq_c, y_seq_c, epochs=100, batch_size=45, validation_split=0.2, callbacks=[early_stopping])

2024-04-25 08:11:05.418897: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 08:11:05.818886: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 08:11:05.819292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 08:11:05.887089: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-25 08:11:06.026980: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-25 08:11:08.129177: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

Epoch 1/100
62/62 [==============================] - 3s 19ms/step - loss: 0.0100 - val_loss: 0.0025
Epoch 2/100
62/62 [==============================] - 1s 10ms/step - loss: 1.6738e-04 - val_loss: 0.0035
Epoch 3/100
62/62 [==============================] - 1s 10ms/step - loss: 1.0281e-04 - val_loss: 0.0022
Epoch 4/100
62/62 [==============================] - 1s 10ms/step - loss: 8.3195e-05 - val_loss: 0.0019
Epoch 5/100
62/62 [==============================] - 1s 10ms/step - loss: 7.0732e-05 - val_loss: 0.0013
Epoch 6/100
62/62 [==============================] - 1s 9ms/step - loss: 6.3838e-05 - val_loss: 0.0014
Epoch 7/100
62/62 [==============================] - 1s 10ms/step - loss: 5.9044e-05 - val_loss: 0.0012
Epoch 8/100
62/62 [==============================] - 1s 12ms/step - loss: 5.5137e-05 - val_loss: 0.0012
Epoch 9/100
62/62 [==============================] - 1s 9ms/step - loss: 5.1575e-05 - val_loss: 0.0011
Epoch 10/100
62/62 [==============================] - 1s 10ms/step - l

In [12]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Build LSTM model
model_h = Sequential([
    LSTM(25, input_shape=(seq_length, num_features_h)),
    Dense(1)
])

# Compile the model
model_h.compile(optimizer='adam', loss='mean_squared_error')

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training the model with early stopping
model_h.fit(X_seq_h, y_seq_h, epochs=100, batch_size=45, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
62/62 [==============================] - 3s 19ms/step - loss: 0.0018 - val_loss: 0.0025
Epoch 2/100
62/62 [==============================] - 1s 8ms/step - loss: 7.2667e-05 - val_loss: 0.0021
Epoch 3/100
62/62 [==============================] - 1s 10ms/step - loss: 4.5729e-05 - val_loss: 0.0015
Epoch 4/100
62/62 [==============================] - 1s 8ms/step - loss: 3.9774e-05 - val_loss: 0.0010
Epoch 5/100
62/62 [==============================] - 1s 10ms/step - loss: 3.5437e-05 - val_loss: 9.2806e-04
Epoch 6/100
62/62 [==============================] - 1s 10ms/step - loss: 3.6167e-05 - val_loss: 9.0233e-04
Epoch 7/100
62/62 [==============================] - 1s 9ms/step - loss: 3.4331e-05 - val_loss: 8.3115e-04
Epoch 8/100
62/62 [==============================] - 1s 11ms/step - loss: 4.1818e-05 - val_loss: 8.8836e-04
Epoch 9/100
62/62 [==============================] - 1s 10ms/step - loss: 2.8176e-05 - val_loss: 8.0552e-04
Epoch 10/100
62/62 [==============================]

In [13]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Build LSTM model
model_l = Sequential([
    LSTM(25, input_shape=(seq_length, num_features_l)),
    Dense(1)
])

# Compile the model
model_l.compile(optimizer='adam', loss='mean_squared_error')

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training the model with early stopping
model_l.fit(X_seq_l, y_seq_l, epochs=100, batch_size=45, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
62/62 [==============================] - 4s 21ms/step - loss: 0.0140 - val_loss: 0.0031
Epoch 2/100
62/62 [==============================] - 1s 12ms/step - loss: 2.8635e-04 - val_loss: 0.0045
Epoch 3/100
62/62 [==============================] - 1s 10ms/step - loss: 1.4115e-04 - val_loss: 0.0033
Epoch 4/100
62/62 [==============================] - 1s 11ms/step - loss: 1.0975e-04 - val_loss: 0.0033
Epoch 5/100
62/62 [==============================] - 1s 11ms/step - loss: 9.0991e-05 - val_loss: 0.0031
Epoch 6/100
62/62 [==============================] - 1s 14ms/step - loss: 8.0833e-05 - val_loss: 0.0028
Epoch 7/100
62/62 [==============================] - 1s 13ms/step - loss: 6.9717e-05 - val_loss: 0.0035
Epoch 8/100
62/62 [==============================] - 1s 12ms/step - loss: 6.2994e-05 - val_loss: 0.0027
Epoch 9/100
62/62 [==============================] - 1s 11ms/step - loss: 5.9198e-05 - val_loss: 0.0032
Epoch 10/100
62/62 [==============================] - 1s 10ms/step -

In [14]:
import pickle

# Serialize and save the model to a file
with open('model_c.pkl', 'wb') as file:
    pickle.dump(model_c, file)

with open('model_h.pkl', 'wb') as file:
    pickle.dump(model_h, file)

with open('model_l.pkl', 'wb') as file:
    pickle.dump(model_l, file)

### data reading:

In [47]:
import os
import re

directory = 'data'

df_data = dict()

for file in os.listdir(directory):
    symbol_pattern = re.match(r'([^_]+)_', file)
    symbol = symbol_pattern.group(1)
    df_data[symbol] = pd.read_csv(os.path.join(directory, file))
    df_data[symbol].rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [48]:
for symbol in df_data:
    from talib import RSI
    df_data[symbol]['rsi_14'] = RSI(df_data[symbol]['close'], timeperiod=14)
    #df_data[symbol]['rsi_14'] = df_data[symbol]['rsi_14'].shift(1)
    
    from talib import MA, SMA, EMA, WMA
    df_data[symbol]['ma_9'] = MA(df_data[symbol]['close'], timeperiod=9)
    #df_data[symbol]['ma_9'] = df_data[symbol]['ma_9'].shift(1)
    df_data[symbol]['sma_9'] = SMA(df_data[symbol]['close'], timeperiod=9)
    #df_data[symbol]['sma_9'] = df_data[symbol]['sma_9'].shift(1)
    df_data[symbol]['wma_9'] = WMA(df_data[symbol]['close'], timeperiod=9)
    #df_data[symbol]['wma_9'] = df_data[symbol]['wma_9'].shift(1)
    
    from talib import MACD
    df_data[symbol]['macd'], df_data[symbol]['signal'], df_data[symbol]['hist'] = MACD(df_data[symbol]['close'])
    #df_data[symbol]['macd'] = df_data[symbol]['macd'].shift(1)
    #df_data[symbol]['signal'] = df_data[symbol]['signal'].shift(1)
    #df_data[symbol]['hist'] = df_data[symbol]['hist'].shift(1)
    
    from talib import ADX
    df_data[symbol]['adx'] = ADX(df_data[symbol]['high'], df_data[symbol]['low'], df_data[symbol]['close'])
    #df_data[symbol]['adx'] = df_data[symbol]['adx'].shift(1)
    
    from talib import ATR
    df_data[symbol]['atr'] = ATR(high=df_data[symbol]['high'], low=df_data[symbol]['low'], close=df_data[symbol]['close'], timeperiod=14)
    #df_data[symbol]['atr'] = df_data[symbol]['atr'].shift(1)
    
    from talib import SAR
    df_data[symbol]['sar'] = SAR(high=df_data[symbol]['high'], low=df_data[symbol]['low'], acceleration=0.02, maximum=0.2)
    #df_data[symbol]['sar'] = df_data[symbol]['sar'].shift(1)
    
    from talib import TEMA
    df_data[symbol]['tema'] = TEMA(df_data[symbol]['close'], timeperiod=14)
    #df_data[symbol]['tema'] = df_data[symbol]['tema'].shift(1)
    
    from talib import ROC
    df_data[symbol]['roc'] = ROC(df_data[symbol]['close'], timeperiod=14)
    #df_data[symbol]['roc'] = df_data[symbol]['roc'].shift(1)
    
    df_data[symbol].dropna(axis=0, inplace=True)

In [49]:
#df_input = dict()
df_pred = dict()

count = 0

for symbol in df_data:

    count += 1
    
    scaler_input = MinMaxScaler()
    data_backup_c_scaled = scaler_input.fit_transform(df_data[symbol][list(best_features_c)])
    scaler_target = MinMaxScaler()
    target_backup_c_scaled = scaler_target.fit_transform(df_data[symbol][['close']])
    X_b_c, y_b_c = create_sequences(data_backup_c_scaled, target_backup_c_scaled, seq_length)
    X_b_c = X_b_c.reshape(X_b_c.shape[0], seq_length, num_features_c)
    #df_input[symbol]['input_c'] = X_b_c

    print(count, symbol)
    y_pred_c = model_c.predict(X_b_c)
    y_pred_c = scaler_target.inverse_transform(y_pred_c)

    scaler_input = MinMaxScaler()
    data_backup_h_scaled = scaler_input.fit_transform(df_data[symbol][list(best_features_h)])
    scaler_target = MinMaxScaler()
    target_backup_h_scaled = scaler_target.fit_transform(df_data[symbol][['high']])
    X_b_h, y_b_h = create_sequences(data_backup_h_scaled, target_backup_h_scaled, seq_length)
    X_b_h = X_b_h.reshape(X_b_h.shape[0], seq_length, num_features_h)
    #df_input[symbol]['input_h'] = X_b_h

    y_pred_h = model_h.predict(X_b_h)
    y_pred_h = scaler_target.inverse_transform(y_pred_h)

    scaler_input = MinMaxScaler()
    data_backup_l_scaled = scaler_input.fit_transform(df_data[symbol][list(best_features_l)])
    scaler_target = MinMaxScaler()
    target_backup_l_scaled = scaler_target.fit_transform(df_data[symbol][['low']])
    X_b_l, y_b_l = create_sequences(data_backup_l_scaled, target_backup_l_scaled, seq_length)
    X_b_l = X_b_l.reshape(X_b_l.shape[0], seq_length, num_features_l)
    #df_input[symbol]['input_l'] = X_b_l

    y_pred_l = model_l.predict(X_b_l)
    y_pred_l = scaler_target.inverse_transform(y_pred_l)

    #dict_data = {'input_c': X_b_c, 'input_h': X_b_h, 'input_l': X_b_l}
    #df_input[symbol] = dict_data
    #print(symbol)

    dict_pred = {'pred_c':y_pred_c, 'pred_h':y_pred_h, 'pred_l':y_pred_l}
    df_pred[symbol] = dict_pred

1 VRTX
156/156 [==============================] - 1s 3ms/step
2 MDLZ
156/156 [==============================] - 1s 3ms/step
3 TMO
156/156 [==============================] - 1s 4ms/step
4 UNH
156/156 [==============================] - 1s 5ms/step
5 TJX
156/156 [==============================] - 1s 4ms/step
6 AAPL
156/156 [==============================] - 1s 4ms/step
7 BKNG
156/156 [==============================] - 1s 5ms/step
8 V
123/123 [==============================] - 0s 3ms/step
9 LOW
156/156 [==============================] - 1s 4ms/step
10 PM
123/123 [==============================] - 0s 3ms/step
11 CMCSA
156/156 [==============================] - 1s 3ms/step
12 HD
156/156 [==============================] - 1s 5ms/step
13 GOOG
151/151 [==============================] - 1s 4ms/step
14 AXP
156/156 [==============================] - 1s 3ms/step
15 NVDA
156/156 [==============================] - 1s 3ms/step
16 SBUX
156/156 [==============================] - 1s 4ms/step
17 PG
156/15

In [50]:
count = 0

for symbol in df_data:
    count += 1
    
    df_data[symbol].reset_index(inplace=True)

    from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error

    score_c = mean_absolute_error(df_data[symbol][['close']].iloc[seq_length:], df_pred[symbol]['pred_c'])
    r2_c = r2_score(df_data[symbol][['close']].iloc[seq_length:], df_pred[symbol]['pred_c'])
    df_data[symbol]['mae_c'] = score_c
    
    score_h = mean_absolute_error(df_data[symbol][['high']].iloc[seq_length:], df_pred[symbol]['pred_h'])
    r2_h = r2_score(df_data[symbol][['high']].iloc[seq_length:], df_pred[symbol]['pred_h'])
    df_data[symbol]['mae_h'] = score_h
    
    score_l = mean_absolute_error(df_data[symbol][['low']].iloc[seq_length:], df_pred[symbol]['pred_l'])
    r2_l = r2_score(df_data[symbol][['low']].iloc[seq_length:], df_pred[symbol]['pred_l'])
    df_data[symbol]['mae_l'] = score_l
    
    print(f'{count} METRICS FOR {symbol}:')
    print(f'score close: {score_c}')
    print(f'score high: {score_h}')
    print(f'score low: {score_l}')

    print(f'- R2 METRICS FOR {symbol}:')
    print(f'r2 close: {r2_c}')
    print(f'r2 high: {r2_h}')
    print(f'r2 low: {r2_l}')

1 METRICS FOR VRTX:
score close: 8.073058508366467
score high: 3.01696394016465
score low: 3.4343664035260915
- R2 METRICS FOR VRTX:
r2 close: 0.9903984090882947
r2 high: 0.9970534082037548
r2 low: 0.9977007543272582
2 METRICS FOR MDLZ:
score close: 0.6284558116192799
score high: 0.5685444529276775
score low: 0.5428047019315053
- R2 METRICS FOR MDLZ:
r2 close: 0.9946362762448319
r2 high: 0.9954464243009632
r2 low: 0.9955411411821588
3 METRICS FOR TMO:
score close: 7.298170739675621
score high: 4.118957081377267
score low: 3.8101911456709407
- R2 METRICS FOR TMO:
r2 close: 0.9969322723874865
r2 high: 0.9979472838876264
r2 low: 0.998939564767169
4 METRICS FOR UNH:
score close: 9.74447541552854
score high: 5.2261998364245565
score low: 3.66919670430532
- R2 METRICS FOR UNH:
r2 close: 0.9939326902986354
r2 high: 0.996885183566923
r2 low: 0.9985395957434197
5 METRICS FOR TJX:
score close: 0.6178801428362068
score high: 0.6881585825877975
score low: 1.1066528933115274
- R2 METRICS FOR TJX:
r

In [51]:
count = 0

remaining_columns = ('date','open','high','low','close','volume','rsi_14','ma_9','mae_c','mae_h','mae_l')

for symbol in df_data:
    count += 1

    df_data[symbol] = df_data[symbol][list(remaining_columns)]

    df_data[symbol]['mpe_c'] = (df_data[symbol]['mae_c'].iloc[-1] / df_data[symbol]['close'].iloc[-1]) * 100
    df_data[symbol]['mpe_h'] = (df_data[symbol]['mae_h'].iloc[-1] / df_data[symbol]['high'].iloc[-1]) * 100
    df_data[symbol]['mpe_l'] = (df_data[symbol]['mae_l'].iloc[-1] / df_data[symbol]['low'].iloc[-1]) * 100
    
    y_pred_c_series = pd.Series(df_pred[symbol]['pred_c'].reshape(1,-1)[0])
    y_pred_h_series = pd.Series(df_pred[symbol]['pred_h'].reshape(1,-1)[0])
    y_pred_l_series = pd.Series(df_pred[symbol]['pred_l'].reshape(1,-1)[0])
    df_data[symbol]['pred_close'] = y_pred_c_series
    df_data[symbol]['pred_highs'] = y_pred_h_series
    df_data[symbol]['pred_lows'] = y_pred_l_series
    df_data[symbol]['pred_close'] = df_data[symbol]['pred_close'].shift(seq_length)
    df_data[symbol]['pred_highs'] = df_data[symbol]['pred_highs'].shift(seq_length)
    df_data[symbol]['pred_lows'] = df_data[symbol]['pred_lows'].shift(seq_length)
    df_data[symbol].to_excel(f'data_w_pred/{symbol}_w_pred.xlsx')

    print(f'{count} done for {symbol}')

1 done for VRTX
2 done for MDLZ
3 done for TMO
4 done for UNH
5 done for TJX
6 done for AAPL
7 done for BKNG
8 done for V
9 done for LOW
10 done for PM
11 done for CMCSA
12 done for HD
13 done for GOOG
14 done for AXP
15 done for NVDA
16 done for SBUX
17 done for PG
18 done for SPGI
19 done for AMZN
20 done for ORCL
21 done for NKE
22 done for CVX
23 done for SYK
24 done for TMUS
25 done for XOM
26 done for LRCX
27 done for AMAT
28 done for HON
29 done for ABNB
30 done for GE
31 done for TSLA
32 done for MRK
33 done for BSX
34 done for UPS
35 done for LLY
36 done for VZ
37 done for T
38 done for MSFT
39 done for CI
40 done for ETN
41 done for JNJ
42 done for PANW
43 done for MDT
44 done for DE
45 done for AVGO
46 done for ELV
47 done for SHOP
48 done for DHR
49 done for BA
50 done for C
51 done for META
52 done for SCHW
53 done for CB
54 done for MCD
55 done for LMT
56 done for BMY
57 done for QCOM
58 done for ABT
59 done for NOW
60 done for IBM
61 done for ISRG
62 done for CAT
63 done

In [37]:
df_data['AAPL'].columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'rsi_14', 'ma_9',
       'mae_c', 'mae_h', 'mae_l', 'mpe_c', 'mpe_h', 'mpe_l', 'pred_close',
       'pred_highs', 'pred_lows'],
      dtype='object')